## Setting up the environment

In [ ]:
# authenticate EE
!earthengine authenticate

In [ ]:
# authenticate and mount Google Drive folder
from google.colab import drive
drive.mount('/content/drive')

In [6]:
# Import the Earth Engine API and initialize it.
import ee
from ee import batch
ee.Initialize()

## Functions

In [17]:
'''
Functions to mask clouds based on the pixel_qa band of Landsat 8 SR data are borrowed from:
https://gis.stackexchange.com/questions/274048/apply-cloud-mask-to-landsat-imagery-in-google-earth-engine-python-api

'''
# function to get QA data from image
def getQABits(image, start, end, mascara):
    # compute the bits we need to extract
    pattern = 0
    for i in range(start,end+1):
        pattern += 2**i
    # return a single band image of the extracted QA bits, giving the band a new name
    return image.select([0],[mascara]).bitwiseAnd(pattern).rightShift(start)

def maskQuality(image):
    # select the QA band
    QA = image.select('pixel_qa')
    # get the internal_cloud_algorithm_flag bit
    sombra = getQABits(QA,3,3,'cloud_shadow')
    nubes = getQABits(QA,5,5,'cloud')
    # detect cirrus clouds using bit 9
    cirrus_detected = getQABits(QA,9,9,'cirrus_detected')
    # return an image masking out cloudy areas
    return image.updateMask(sombra.eq(0)).updateMask(nubes.eq(0).updateMask(cirrus_detected.eq(0)))


## Spatial and Temporal Filters

In [8]:
# create area of interest polygon (several options presented below)
## Example 1: Simple bounding box for the entire Gulf of Mexico
# aoipoly = ee.Geometry.BBox(-98, 26,-82,31)

## Example 2: Bounding polygon for area around Pensacola, FL, USA
aoipoly = ee.Geometry.Polygon([[[-88.412770, 31.098514], 
                                [-87.015281, 31.098514], 
                                [-87.015281, 30.156911], 
                                [-88.412770, 30.156911], 
                                [-88.412770, 31.098514]]])

## Example 3: Bounding polygon for entire Gulf of Mexico (USA) coast
# aoipoly = ee.Geometry.Polygon([[[-98.347274, 26.322097], 
#                                 [-98.238364, 28.187734], 
#                                 [-95.430534, 32.017493], 
#                                 [-84.011970, 32.564172], 
#                                 [-82.679402, 31.166281], 
#                                 [-80.410998, 24.154884], 
#                                 [-86.730146, 28.166929], 
#                                 [-94.552293, 27.623251], 
#                                 [-96.338964, 25.553449], 
#                                 [-98.347274, 26.322097]]])

# start and end dates for filtering imagery
START_DATE = '2016-01-01'
END_DATE = '2021-04-30'

## Download Landsat 8 Tier 1 imagery

In [147]:
# try to load an EE image collection
try:
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
except:  # if the loading image collection fails, then initialize EE in the code
    ee.Initialize()

# specify the image collection (Landsat 8), including its space and time filters
dataset = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
                .filterDate(START_DATE, END_DATE)\
                .filterBounds(aoipoly)\
                .map(maskQuality)

# bulk/batch export the download imagery to Google Drive
task = ee.batch.Export.image.toDrive(image = dataset.mean(),
                                     description = 'Landsat8_Tier1_imagery',
                                     folder = 'delta_dat',
                                     region = aoipoly,
                                     scale = 30,
                                     fileNamePrefix = 'L8T1_GoM')

In [ ]:
# start data download
task.start()

In [ ]:
# check the status of the data download
task.status()